This notebook is related to work on the [https://github.com/adellej/beans](beans) code for matching observations of thermonuclear bursts to models. 

The notebook specifically will run settle for a set of cases for which Kepler runs also exist, for a comparison of the outputs, as well as potential benchmarking.

In [7]:
import os
import sys

# add your path to beans here

sys.path.append('/home/martin/src/CIC/Adele/beans/beans')
sys.path
    
from astropy.table import Table
from astropy.io import ascii
import astropy.units as u

from beans import settle

## Comparison of `settle` with Kepler

Here we want to use the `concord` table and run a `settle` model with each set to compare the results. First need to read in the table

In [48]:
# add your path to the mrt file below

ft = ascii.read('table1.mrt')
print("ft length = ", len(ft))
# fixed parameters
M_NS, R_NS, Q_b = 1.4, 10., 0.1
# print (M_NS, R_NS)
# for run in ft:
#     print (run['run'], run['mdot'], run['X'], run['Z'])
    

ft length =  60


In [59]:
# This section was run on xray, where settle works

tdel, E_b, alpha = [], [], []

import time
t_start = time.process_time()
t1_sum = 0.0
t2_sum = 0.0

num_runs=20

for j in range(num_runs):
    t2_start = time.process_time()
    for i, run in enumerate(ft['run']):
        ### print ('Running settle for run #{}...'.format(run))
        # need to convert the mdot here, I think this is right
        # In the MRT file accretion rate is given as a fraction of the Eddington rate, i.e.  
        # Mdot_Edd = 8.8e4/(1+X) g/cm^2/s; and since settle uses fraction of 8.8e4, we have
        # an extra factor of (1+X) in the MRT values that we need to divide by
        t1_start = time.process_time()
        res = settle(Q_b, ft[i]['Z'], ft[i]['X'], ft[i]['mdot']/(1+ft[i]['X']), 1.0, M_NS, R_NS)
        t1_end = time.process_time()
        t1_sum += (t1_end-t1_start)
        tdel.append(res['tdel'][0])
        E_b.append(res['E_b'][0])
        alpha.append(res['alpha'][0])
    t2_end = time.process_time()
    t2_sum += (t2_end-t2_start)
    print("Cycle #", j, " time = ", (t2_end-t2_start))

t_end = time.process_time()
print("total process time (",num_runs, "loops) = ", t_end - t_start)
print("settle sum time (",num_runs, "loops) = ", t1_sum)
print("loop sum time (",num_runs, "loops) = ", t2_sum)
print("average", len(ft), "row data table one loop run settle sum time = ", t1_sum/num_runs)
    
t = Table([ft['run'], tdel[ : len(ft)], E_b[ : len(ft)], alpha[ : len(ft)]])
t.write('settle.txt', format='ascii', overwrite=True)

Cycle # 0  time =  1.0859557230000405
Cycle # 1  time =  1.1577557979999824
Cycle # 2  time =  1.0331596110000305
Cycle # 3  time =  1.0980430100000262
Cycle # 4  time =  1.0633876910000026
Cycle # 5  time =  1.083188716000052
Cycle # 6  time =  1.022214765000001
Cycle # 7  time =  1.0456627119999666
Cycle # 8  time =  1.0444267509999463
Cycle # 9  time =  1.0586430849999715
Cycle # 10  time =  1.0277684799999633
Cycle # 11  time =  1.0773964730000216
Cycle # 12  time =  1.0260536099999626
Cycle # 13  time =  1.0722670440000002
Cycle # 14  time =  1.0709616890000007
Cycle # 15  time =  1.1106033569999454
Cycle # 16  time =  1.0623965119999639
Cycle # 17  time =  1.0466508040000235
Cycle # 18  time =  1.055228032000059
Cycle # 19  time =  1.0195906459999833
total process time ( 20 loops) =  21.263548350000065
settle sum time ( 20 loops) =  21.232073904000515
loop sum time ( 20 loops) =  21.261354508999943
average 60 row data table one loop run settle sum time =  1.0616036952000258
